In [1]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install tensorflow
!pip install emoji
# !pip install farasa

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from tqdm import tqdm
from preprocessing import preprocess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add

C:\Users\Asus\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
#path = '/content/drive/MyDrive/NN Arabic Sentiment Analysis'
train_data = pd.read_excel('train.xlsx')
train_data['rating'] = train_data['rating'] + 1
test_data = pd.read_csv('test _no_label.csv')

train_data['review_description'] = train_data['review_description'].apply(lambda x: preprocess(x, remove_mode=1, emoji_mode=0, token_mode=0, stem_mode=0))
test_data['review_description'] = test_data['review_description'].apply(lambda x: preprocess(x, remove_mode=1, emoji_mode=0, token_mode=0, stem_mode=0))

# Subset the data
subset_size = len(train_data)
train_data_subset = train_data.sample(n=subset_size, random_state=42)

# Prepare features and labels
features = train_data_subset['review_description']
labels = train_data_subset['rating'].values.astype(np.int64)
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(features)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(features)
# Pad sequences
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [5]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, encoded_labels, test_size=0.2, random_state=42
)

In [6]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.positional_encoding = self.positional_encoding(position, embed_dim)

    def get_angles(self, position, i, embed_dim):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(embed_dim, tf.float32))
        return position / angles

    def positional_encoding(self, position, embed_dim):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(embed_dim, dtype=tf.float32)[tf.newaxis, :],
            embed_dim=embed_dim,
        )
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.positional_encoding[:, : tf.shape(inputs)[1], :]

In [ ]:
# Model parameters
embed_dim = 512
num_heads = 3
ff_dim = 32
num_transformer_blocks = 1

# Build the transformer model
inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
x = embedding_layer(inputs)

# Add Bidirectional LSTM
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embed_dim * 2, return_sequences=True))(x)  # Use embed_dim as the number of units

# Add a Dense layer
x = tf.keras.layers.Dense(embed_dim, activation='relu')(x)

# Add Positional Encoding
positional_encoding = PositionalEncoding(max_len, embed_dim)(x)

# Transformer blocks
for _ in range(num_transformer_blocks):
    # Multi-Head Attention
    attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(positional_encoding, positional_encoding, positional_encoding)
    attention = tf.keras.layers.Dropout(0.1)(attention)
    add_attention = tf.keras.layers.Add()([positional_encoding, attention])
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_attention)

    # Feed Forward Part
    ffn = tf.keras.layers.Dense(ff_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(attention)
    ffn = tf.keras.layers.Dense(embed_dim)(ffn)
    ffn = tf.keras.layers.Dropout(0.1)(ffn)
    add_ffn = tf.keras.layers.Add()([attention, ffn])
    positional_encoding = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_ffn)

x = tf.keras.layers.GlobalAveragePooling1D()(positional_encoding)
outputs = tf.keras.layers.Dense(3, activation="softmax")(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

# Compile the model with customized optimizer
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")





  6/801 [..............................] - ETA: 1:36:31 - loss: 1.0173 - accuracy: 0.4375